In [35]:
import numpy as np
import pandas as pd

# Raw Data Loding
train = pd.read_csv('./data/titanic/train.csv')

# PassengerId : 승객일련번호
#               필요 없을 거 같다.

# Survived : 생존여부 0: 사망, 1: 생존
#            label, 당연히 포함시켜야 한다.

# Pclass : 객실 등급 (1등급, 2등급, 3등급)
#          상관계수를 구해보고 연관성을 파악한 후 아마도 살리는게 좋을거 같다.

# Name : 이름이 필요 없을거 같은데 ... 사용함 ->  age 채우고 버림

# Age : 나이 연관성이 있어서 살려야 한다. - 결측치가 있어서 어떻게 처리할지 고민필요
#      가장 쉬운 방법은 : 모든 사람의 나이 평균 => 별로...
#       그 사람 이름에 포함된 MR,MRS,MISS를 따져서 그 분류의 평균 나이 사용
#       유아(0) 청소년(1) 성인(2) 노인(3)

# Sex : male, female => 0, 1로 변경

# SibSp, Parch: 두 column을 합쳐서 num_of_family 만들어서 사용.

# Ticket: 삭제
# Fare : 요금... 고민을 해 봐야한다.

# cabin : 객실번호 x

# Embarked : 어디에서 승선했는지 => S,C,Q => 숫자로 바꾸어서 처리

# pandas를 이용해서 학습에 적합한 형태로 데이터를 처리해보자
# map_sex = {'male':0, 'female':1}
# map_Emba = {'S':0, 'C':1, 'Q':2}
# df['num_of_family'] = df['SibSp'] + df['Parch']
# df[['Sex']] = df[['Sex']].applymap(map_sex.get)
# df[['Embarked']] = df[['Embarked']].applymap(map_Emba.get)
# df = df.drop(['PassengerId','Ticket','Cabin', 'SibSp', 'Parch'], axis=1)
# display(df)

# 필요없는 column부터 제거해보자
train.drop(['PassengerId', 'Name', 'Ticket', 'Fare', 'Cabin'], axis=1, inplace=True)

# 성별처리
sex_mapping = { 'male' : 0, 'female' : 1 }
train['Sex'] = train['Sex'].map(sex_mapping)

# 가족처리
train['Family'] = train['SibSp'] + train['Parch']
train.drop(['SibSp', 'Parch'], axis=1, inplace=True)

# Embarked 결측치 처리
train['Embarked'] = train['Embarked'].fillna('Q')

# Age에 대한 결측치 처리
train['Age'] = train['Age'].fillna(train['Age'].mean())

# Embarked 문자 -> 숫자 처리
embarked_mapping = {'S' : 0 , 'C': 1, 'Q': 2 }
train['Embarked'] = train['Embarked'].map(embarked_mapping)

# Age에 대해서 Binning 처리 (Numerical value -> categorical value)
train.loc[train['Age'] < 8, 'Age'] = 0
train.loc[(train['Age'] >= 8) & (train['Age'] < 20), 'Age'] = 1
train.loc[(train['Age'] >=20) & (train['Age'] < 65 ), 'Age'] = 2
train.loc[train['Age'] >=65, 'Age'] = 3

display(train)

# 데이터가 준비됐다.
# Tensorflow를 이용해서 학습을 진행하고 Accuracy를 구해보자
# 891개의 데이터가 있다. 이걸 다 학습에 사용하면 안된다.
# 7:3 비율로 데이터를 나누어서 학습과 정확도 측정에 사용하자



,Survived,Pclass,Sex,Age,Embarked,Family
0,0,3,0,2.0,0,1
1,1,1,1,2.0,1,1
2,1,3,1,2.0,0,0
3,1,1,1,2.0,0,1
4,0,3,0,2.0,0,0
...,...,...,...,...,...,...
886,0,2,0,2.0,0,0
887,1,1,1,1.0,0,0
888,0,3,1,2.0,0,3
889,1,1,0,2.0,1,0


In [14]:
# Multinomial Classification
# BMI 지수로 학습해보자. => 키와 몸무게를 가지고 저체중, 정상 과체중 비만을 판단하는 지수
# BMI = 자신의 몸무게(kg) / 키의 제곱(m)
#      18.5 이하 => 저체중
#      18.5 ~ 23 => 정상
#      23 ~ 25 => 과체중
#      25 ~ => 비만
# 우리가 하려는 건 식이 아니라 BMI 지수를 조사한 데이터가 있다.
# 이걸 학습해서 예측을 통해 나의 BMI 지수를 알아보자
# 단 제공하는 데이터는 4가지가 아니라 3가지 분류로 되어있다.

import numpy as np
import pandas as pd
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

df = pd.read_csv('./data/bmi/bmi.csv', skiprows=3)

# display(df)

# 결측치 확인
# df.isnull().sum()        # 결측치 없음

# 이상치 확인
zscore = 1.8

# 이상치를 확인
# df.loc[np.abs(stats.zscore(df['height'])) >= zscore, :] # height의 이상치는 없다
# df.loc[np.abs(stats.zscore(df['weight'])) >= zscore, :] # weight의 이상치는 없다
# df.loc[np.abs(stats.zscore(df['label'])) >= zscore, :] # label의 이상치는 없다

# Data Split
# Train, Test 두 부분으로 분할. 분리하는 비율은 7:3으로 분리
# 나중에 Train부분은 k-fold cross validation을 진행
x_data_train, x_data_test, t_data_train, t_data_test = \
train_test_split(df[['height','weight']],df['label'],test_size = 0.3, random_state=0) # 14000 / 6000

# Normalization
scaler = MinMaxScaler()  # scaler 객체를 생성
scaler.fit(x_data_train) # scaler 객체에 최대 최소와 같은 정보가 들어간다. (fit 처리)

x_data_train_norm = scaler.transform(x_data_train)
x_data_test_norm = scaler.transform(x_data_test)

del x_data_train       # 혼동 방지를 위해 변수를 삭제
del x_data_test

# sklearn 구현은 매우매우 간단 - model 생성하고 학습진행
model = LogisticRegression()
model.fit(x_data_train_norm, t_data_train)

# 우리 model의 정확도를 측정해야한다.
# cross validation
kfold = 10
kfold_score = cross_val_score(model,x_data_train_norm, t_data_train, cv=kfold)
print('### cross validation ###')
print('score : {}'.format(kfold_score))
print('평균: {}'.format(kfold_score.mean()))

# 최종모델평가
predict_val = model.predict(x_data_test_norm)  # 테스트 데이터로 예측값을 구해요
acc = accuracy_score(predict_val, t_data_test)

print('우리 Model의 최종 Accuracy : {}'.format(acc))

# Predict

height = 188
weight = 78
my_state = [[height,weight]]
my_state_val = model.predict(scaler.transform(my_state))
print(my_state_val)

### cross validation ###
score : [0.98       0.98642857 0.985      0.97642857 0.98642857 0.98428571
 0.98714286 0.97714286 0.97714286 0.98642857]
평균: 0.9826428571428572
우리 Model의 최종 Accuracy : 0.9845
[1]
